In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# LOAD DATA
# =============================================================================

print("="*80)
print("NYC HOUSING DATA PROCESSING")
print("="*80)

df = pd.read_csv('/Volumes/workspace/default/nyc-housing/nyc_housing_base.csv')
print(f"Loaded {len(df):,} records")

# =============================================================================
# CONSTANTS
# =============================================================================

BOROUGH_NAMES = {
    1: 'Manhattan',
    2: 'Bronx',
    3: 'Brooklyn',
    4: 'Queens',
    5: 'Staten Island'
}

BUILDING_CLASS_NAMES = {
    'A0': 'One Family - Cape Cod',
    'A1': 'One Family - Two Story Detached',
    'A2': 'One Family - One Story',
    'A3': 'One Family - Large Suburban',
    'A4': 'One Family - City Residence',
    'A5': 'One Family - Attached/Semi-Detached',
    'A6': 'One Family - Summer Cottage',
    'A7': 'One Family - Mansion',
    'A9': 'One Family - Miscellaneous',
    'B1': 'Two Family - Brick',
    'B2': 'Two Family - Frame',
    'B3': 'Two Family - Converted',
    'B9': 'Two Family - Miscellaneous',
    'C0': 'Walk-Up - Three Families',
    'C1': 'Walk-Up - Over 6 Units',
    'C2': 'Walk-Up - 5-6 Units',
    'C3': 'Walk-Up - 4 Units',
    'C4': 'Walk-Up - Old Law Tenement',
    'C5': 'Walk-Up - Converted',
    'C6': 'Walk-Up - Cooperative',
    'C7': 'Walk-Up - With Commercial',
    'C8': 'Walk-Up - Co-op With Commercial',
    'C9': 'Walk-Up - Garden Apartment',
    'D0': 'Elevator - Co-op',
    'D1': 'Elevator - Semi-Fireproof',
    'D2': 'Elevator - Artists In Residence',
    'D3': 'Elevator - Fireproof With Stores',
    'D4': 'Elevator - Cooperative',
    'D5': 'Elevator - Converted',
    'D6': 'Elevator - Fireproof',
    'D7': 'Elevator - With Commercial',
    'D8': 'Elevator - Co-op With Commercial',
    'D9': 'Elevator - Miscellaneous',
    'R1': 'Condo - Residential',
    'R2': 'Condo - With Garage',
    'R3': 'Condo - Professional Unit',
    'R4': 'Condo - Commercial',
    'R6': 'Condo - Garage/Parking',
    'R9': 'Condo - Miscellaneous',
    'S0': 'Mixed Use - 1 Story Store',
    'S1': 'Mixed Use - 2 Story Store',
    'S2': 'Mixed Use - 3 Story Store',
    'S3': 'Mixed Use - 4 Story Store',
    'S4': 'Mixed Use - 5+ Story Store',
    'S5': 'Mixed Use - Store With Apartments',
    'S9': 'Mixed Use - Miscellaneous',
    'K1': 'Retail - One Story',
    'K2': 'Retail - Two Story',
}

BUILDING_CATEGORY = {
    'A': 'Single Family Home',
    'B': 'Two Family Home',
    'C': 'Walk-Up Apartment',
    'D': 'Elevator Apartment',
    'R': 'Condo',
    'S': 'Mixed Use',
    'K': 'Retail',
    'O': 'Office',
    'E': 'Warehouse',
    'F': 'Factory/Industrial',
    'G': 'Garage/Parking',
    'H': 'Hotel',
    'L': 'Loft',
    'M': 'Religious',
    'V': 'Vacant Land',
    'W': 'School/Education',
    'I': 'Healthcare',
    'J': 'Theater',
    'N': 'Institutional',
    'P': 'Recreation',
    'Q': 'Parks',
    'T': 'Transportation',
    'U': 'Utility',
    'Y': 'Government',
    'Z': 'Mixed Use (Other)',
}

# =============================================================================
# FUNCTIONS
# =============================================================================

def assign_borough(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return 'Unknown'
    if lon < -74.05:
        return 'Staten Island'
    if lat >= 40.79 and lon > -74.05:
        return 'Bronx'
    if 40.70 <= lat <= 40.88 and -74.02 <= lon <= -73.93:
        return 'Manhattan'
    if lat < 40.70 and lon > -74.05 and lon < -73.85:
        return 'Brooklyn'
    if lat < 40.74 and lon <= -73.92:
        return 'Brooklyn'
    if lon >= -73.88:
        return 'Queens'
    if lat < 40.74:
        return 'Brooklyn'
    return 'Queens'

def get_building_category(bldg_class):
    if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
        return 'Unknown'
    return BUILDING_CATEGORY.get(str(bldg_class)[0].upper(), 'Other')

def get_use_type(bldg_class):
    if pd.isna(bldg_class) or len(str(bldg_class)) == 0:
        return 'Unknown'
    first = str(bldg_class)[0].upper()
    if first in ['A', 'B', 'C', 'D', 'R']:
        return 'Residential'
    elif first in ['O', 'K', 'E', 'F', 'G', 'H', 'I', 'J', 'L']:
        return 'Commercial'
    elif first in ['S', 'Z']:
        return 'Mixed Use'
    return 'Special Purpose'

# =============================================================================
# PROCESSING
# =============================================================================

print("\n🔧 Processing...")

# Convert numeric columns
numeric_cols = ['sale_price', 'bldgarea', 'lotarea', 'resarea', 'comarea',
                'yearbuilt', 'building_age', 'numfloors', 'unitsres', 
                'unitstotal', 'latitude', 'longitude']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Remove invalid building areas
initial = len(df)
df = df[~((df['bldgarea'].notna()) & (df['bldgarea'] <= 0))].copy()
print(f"  Removed {initial - len(df):,} invalid building area records")

# Assign boroughs
has_coords = df['latitude'].notna() & df['longitude'].notna()
df.loc[has_coords, 'borough_name'] = df[has_coords].apply(
    lambda row: assign_borough(row['latitude'], row['longitude']), axis=1
)

if 'borough_x' in df.columns:
    no_coords = df['borough_name'].isna()
    df.loc[no_coords, 'borough_name'] = df.loc[no_coords, 'borough_x'].map(BOROUGH_NAMES)

df['borough_name'] = df['borough_name'].fillna('Unknown')

# Building classifications
df['building_type'] = df['bldgclass'].map(BUILDING_CLASS_NAMES).fillna('Other')
df['building_category'] = df['bldgclass'].apply(get_building_category)
df['use_type'] = df['bldgclass'].apply(get_use_type)

# =============================================================================
# CALCULATE PRICE PER SQFT - PROPERLY
# =============================================================================

print("\n📊 Calculating price per sqft...")

# Calculate raw price per sqft
df['price_per_sqft_raw'] = np.where(
    df['bldgarea'] > 0,
    df['sale_price'] / df['bldgarea'],
    np.nan
)

# Flag for valid price per sqft (reasonable range for NYC)
# Residential: $100 - $3000/sqft is typical
# Below $100 = likely whole-building, commercial bulk, or data issue
# Above $3000 = likely data error or ultra-luxury outlier
df['valid_pps'] = (
    (df['price_per_sqft_raw'] >= 100) & 
    (df['price_per_sqft_raw'] <= 3000) &
    (df['use_type'] == 'Residential')
)

# Create filtered price per sqft column
df['price_per_sqft'] = np.where(df['valid_pps'], df['price_per_sqft_raw'], np.nan)

# For residential analysis, also filter by unit size (exclude whole buildings)
df['is_unit_sale'] = (
    (df['bldgarea'] > 200) &  # At least 200 sqft
    (df['bldgarea'] < 10000) &  # Less than 10,000 sqft (single unit)
    (df['sale_price'] > 50000) &  # Real sale
    (df['use_type'] == 'Residential')
)

# =============================================================================
# CREATE CATEGORIES
# =============================================================================

print("📋 Creating categories...")

# Price tiers
df['price_tier'] = pd.cut(
    df['sale_price'],
    bins=[0, 300000, 600000, 900000, 1500000, 3000000, float('inf')],
    labels=['Under $300K', '$300K-600K', '$600K-900K', '$900K-1.5M', '$1.5M-3M', 'Over $3M']
)

# Size categories (for residential units)
df['size_category'] = pd.cut(
    df['bldgarea'],
    bins=[0, 500, 800, 1200, 2000, 3500, float('inf')],
    labels=['Studio (<500)', 'Small (500-800)', '1BR (800-1200)', 
            '2BR (1200-2000)', '3BR+ (2000-3500)', 'Large (3500+)']
)

# Age categories
df['age_category'] = pd.cut(
    df['building_age'],
    bins=[0, 10, 25, 50, 75, 100, float('inf')],
    labels=['New (0-10)', 'Recent (10-25)', 'Modern (25-50)', 
            'Mature (50-75)', 'Old (75-100)', 'Historic (100+)']
)

# =============================================================================
# SUMMARY
# =============================================================================

print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)

print(f"\nTotal records: {len(df):,}")
print(f"Records with valid coordinates: {has_coords.sum():,}")
print(f"Records with valid price/sqft: {df['valid_pps'].sum():,}")
print(f"Residential unit sales: {df['is_unit_sale'].sum():,}")

print(f"\n📊 Price Statistics:")
print(f"  Median: ${df['sale_price'].median():,.0f}")
print(f"  Mean:   ${df['sale_price'].mean():,.0f}")

valid_pps = df[df['valid_pps']]['price_per_sqft']
if len(valid_pps) > 0:
    print(f"\n📊 Price/SqFt (Residential Units $100-$3000 range):")
    print(f"  Records: {len(valid_pps):,}")
    print(f"  Median:  ${valid_pps.median():,.0f}")
    print(f"  Mean:    ${valid_pps.mean():,.0f}")

print(f"\n🏘️ Borough Distribution:")
for borough, count in df['borough_name'].value_counts().items():
    pct = count / len(df) * 100
    # Get median price/sqft for this borough
    borough_pps = df[(df['borough_name'] == borough) & df['valid_pps']]['price_per_sqft']
    pps_str = f"${borough_pps.median():,.0f}/sqft" if len(borough_pps) > 0 else "N/A"
    print(f"  {borough:<20} {count:>8,} ({pct:>5.1f}%)  Median: {pps_str}")

# =============================================================================
# SAVE
# =============================================================================

df.to_csv('nyc_housing_processed.csv', index=False)
print(f"\n💾 Saved: nyc_housing_processed.csv")

# Save residential only
df_res = df[df['is_unit_sale']].copy()
df_res.to_csv('nyc_housing_residential_units.csv', index=False)
print(f"💾 Saved: nyc_housing_residential_units.csv ({len(df_res):,} records)")

print("\n✅ Data processing complete!")